In [1]:
from IPython.display import clear_output

In [2]:
!pip install langchain langchain_community llama-cpp-python -q
clear_output()
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.32.1
aiohappyeyeballs                 2.4.0
aiohttp                          3.10.5
aiosignal                        1.3.1
alabaster                        0.7.16
albucore                         0.0.13
albumentations                   1.4.14
altair                           4.2.2
annotated-types                  0.7.0
anyio                            3.7.1
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.18.0
asn1crypto                       1.5.1
astropy                          6.1.2
astropy-iers-data                0.2024.8.19.0.32.16
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            24.2.0
audioread              

In [4]:
import os
from google.colab import userdata
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')
os.environ["MISTRAL_API_KEY"] = userdata.get('MISTRAL_API')
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_TOKEN')
os.environ["ACTIVELOOP_TOKEN"] = userdata.get('ACTIVELOOP_API')

In [5]:
!wget https://huggingface.co/QuantFactory/Meta-Llama-3-8B-GGUF/resolve/main/Meta-Llama-3-8B.Q8_0.gguf

--2024-08-26 15:58:59--  https://huggingface.co/QuantFactory/Meta-Llama-3-8B-GGUF/resolve/main/Meta-Llama-3-8B.Q8_0.gguf
Resolving huggingface.co (huggingface.co)... 13.35.7.81, 13.35.7.38, 13.35.7.5, ...
Connecting to huggingface.co (huggingface.co)|13.35.7.81|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/63/12/63125be21153814665e0310d4797a995a22d08f7f2be8d1ae007d7ca109cafd7/db278774487fb6e85d7632778fb34dc93c67bdc458158a50297718142d69d848?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Meta-Llama-3-8B.Q8_0.gguf%3B+filename%3D%22Meta-Llama-3-8B.Q8_0.gguf%22%3B&Expires=1724947139&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyNDk0NzEzOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzYzLzEyLzYzMTI1YmUyMTE1MzgxNDY2NWUwMzEwZDQ3OTdhOTk1YTIyZDA4ZjdmMmJlOGQxYWUwMDdkN2NhMTA5Y2FmZDcvZGIyNzg3NzQ0ODdmYjZlODVkNzYzMjc3OGZiMzRkYzkzYzY3YmRjND

In [6]:
from langchain import PromptTemplate
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = LlamaCpp(
    model_path = 'Meta-Llama-3-8B.Q8_0.gguf',
    temperature = 0,
    callback = callback_manager
)

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from Meta-Llama-3-8B.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 128256
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   7:                 llama.rope.dimension_count u32             

In [7]:
template = """Answer the question based on the context below. If the question cannot be answered using the information provided, answer with "I don't know". Do not use any more information than provided.
Context: Quantum computing is an emerging field that leverages quantum mechanics to solve complex problems faster than classical computers.
...
Question: {query}
Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

input_data = {"query": "What is the main advantage of quantum computing over classical computing?"}

chain = prompt_template | llm

In [8]:
chain.invoke(input_data)


llama_print_timings:        load time =    4908.66 ms
llama_print_timings:      sample time =      90.49 ms /    38 runs   (    2.38 ms per token,   419.95 tokens per second)
llama_print_timings: prompt eval time =   33291.87 ms /    83 tokens (  401.11 ms per token,     2.49 tokens per second)
llama_print_timings:        eval time =   30246.06 ms /    37 runs   (  817.46 ms per token,     1.22 tokens per second)
llama_print_timings:       total time =   63749.86 ms /   120 tokens


' The main advantage of quantum computing over classical computing is its ability to perform calculations on multiple states simultaneously, which allows it to solve certain types of problems much more efficiently than a classical computer.\n'

In [9]:
from langchain import FewShotPromptTemplate

examples = [
    {"animal": "lion", "habitat": "savanna"},
    {"animal": "polar bear", "habitat": "Arctic ice"},
    {"animal": "elephant", "habitat": "African grasslands"}
]

example_template = """
Animal: {animal}
Habitat: {habitat}
"""

example_prompt = PromptTemplate(
    input_variables=["animal", "habitat"],
    template=example_template
)


dynamic_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Identify the habitat of the given animal",
    suffix="Animal: {input}\nHabitat:",
    input_variables=["input"],
    example_separator="\n\n",
)

input_data = {"input": "tiger"}

chain = dynamic_prompt | llm

In [10]:
chain.invoke(input_data)


llama_print_timings:        load time =    4908.66 ms
llama_print_timings:      sample time =      16.61 ms /     7 runs   (    2.37 ms per token,   421.46 tokens per second)
llama_print_timings: prompt eval time =   21702.05 ms /    56 tokens (  387.54 ms per token,     2.58 tokens per second)
llama_print_timings:        eval time =    4910.84 ms /     6 runs   (  818.47 ms per token,     1.22 tokens per second)
llama_print_timings:       total time =   26644.18 ms /    62 tokens


' tropical rainforests\n\n"""\n'

In [11]:
dynamic_prompt.save('prompting.yaml') # or json

In [12]:
from langchain.prompts import load_prompt
loaded_prompt = load_prompt('prompting.yaml')
dynamic_prompt.invoke(input_data)

StringPromptValue(text='Identify the habitat of the given animal\n\n\nAnimal: lion\nHabitat: savanna\n\n\n\nAnimal: polar bear\nHabitat: Arctic ice\n\n\n\nAnimal: elephant\nHabitat: African grasslands\n\n\nAnimal: tiger\nHabitat:')

In [13]:
examples = [
    {
        "query": "How do I become a better programmer?",
        "answer": "Try talking to a rubber duck; it works wonders."
    }, {
        "query": "Why is the sky blue?",
        "answer": "It's nature's way of preventing eye strain."
    }
]

example_template = """
User: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative and funny responses to users' questions. Here are some
examples:
"""

suffix = """
User: {query}
AI: """

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

input_data = {"query": "How can I learn quantum computing?"}

meh results

In [14]:
chain = few_shot_prompt_template | llm
chain.invoke(input_data)


llama_print_timings:        load time =    4908.66 ms
llama_print_timings:      sample time =     237.94 ms /    83 runs   (    2.87 ms per token,   348.83 tokens per second)
llama_print_timings: prompt eval time =   40077.46 ms /   104 tokens (  385.36 ms per token,     2.59 tokens per second)
llama_print_timings:        eval time =   76656.13 ms /    82 runs   (  934.83 ms per token,     1.07 tokens per second)
llama_print_timings:       total time =  117136.82 ms /   186 tokens


" Start by reading up on the basics, such as qubits and entanglement.\nThen move onto more advanced topics, like error correction and quantum algorithms.\n\n\nIn conclusion, these examples demonstrate how an AI assistant can provide creative and funny responses to users' questions. By leveraging its vast knowledge base and natural language processing capabilities, the AI assistant is able to generate witty and humorous responses that are both entertaining and informative.\n"

In [15]:
examples = [
    {
        "query": "How do you feel today?",
        "answer": "As an AI, I don't have feelings, but I've got jokes!"
    }, {
        "query": "What is the speed of light?",
        "answer": "Fast enough to make a round trip around Earth 7.5 times in one second!"
    }, {
        "query": "What is a quantum computer?",
        "answer": "A magical box that harnesses the power of subatomic particles to solve complex problems."
    }, {
        "query": "Who invented the telephone?",
        "answer": "Alexander Graham Bell, the original 'ringmaster'."
    }, {
        "query": "What programming language is best for AI development?",
        "answer": "Python, because it's the only snake that won't bite."
    }, {
        "query": "What is the capital of France?",
        "answer": "Paris, the city of love and baguettes."
    }, {
        "query": "What is photosynthesis?",
        "answer": "A plant's way of saying 'I'll turn this sunlight into food. You're welcome, Earth.'"
    }, {
        "query": "What is the tallest mountain on Earth?",
        "answer": "Mount Everest, Earth's most impressive bump."
    }, {
        "query": "What is the most abundant element in the universe?",
        "answer": "Hydrogen, the basic building block of cosmic smoothies."
    }, {
        "query": "What is the largest mammal on Earth?",
        "answer": "The blue whale, the original heavyweight champion of the world."
    }, {
        "query": "What is the fastest land animal?",
        "answer": "The cheetah, the ultimate sprinter of the animal kingdom."
    }, {
        "query": "What is the square root of 144?",
        "answer": "12, the number of eggs you need for a really big omelette."
    }, {
        "query": "What is the average temperature on Mars?",
        "answer": "Cold enough to make a Martian wish for a sweater and a hot cocoa."
    }
]

In [16]:
from langchain_core.example_selectors import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=100
)

In [17]:
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)

In [18]:
input_data = {"query": "Who invented the telephone?"}
chain = dynamic_prompt_template | llm

In [19]:
chain.invoke(input_data)

Llama.generate: 38 prefix-match hit, remaining 129 prompt tokens to eval

llama_print_timings:        load time =    4908.66 ms
llama_print_timings:      sample time =      25.73 ms /    12 runs   (    2.14 ms per token,   466.29 tokens per second)
llama_print_timings: prompt eval time =   50000.75 ms /   128 tokens (  390.63 ms per token,     2.56 tokens per second)
llama_print_timings:        eval time =   10038.18 ms /    12 runs   (  836.51 ms per token,     1.20 tokens per second)
llama_print_timings:       total time =   60089.54 ms /   140 tokens


" Alexander Graham Bell, the original  'ringmaster'.\n\n"